In [1]:
import sys
sys.path.append('/Users/yasaman/UWEXP/cleanup-scripts')
from jobutils import utils
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_path = '/Users/yasaman/Downloads/aggregatedcounts/merged'
table_file = '/Users/yasaman/UWEXP/script-input/tables.csv'
participant_file = '/Users/yasaman/UWEXP/script-input/participants.csv'
result_path = '/Users/yasaman/Downloads/aggregatedcounts/figures_merged/'

In [3]:
tables = utils.readList(table_file)
participants = utils.readList(participant_file)

In [4]:
def load_data(data_file):
    data = pd.read_csv(data_file,
                       header=0,
                       index_col=0,
                       sep='\t', 
                       lineterminator='\n',
                       encoding = "ISO-8859-1")
    return data

def plot_data(data, tableORpid, xticklabels, yticklabels, width=None, height=None):
    if width is None or height is None:
        fig, ax = plt.subplots()
    else:
        fig, ax = plt.subplots(figsize=(width, height))
    sns.heatmap(data.transpose().isnull(), 
                cbar=False,
                xticklabels=xticklabels,
                yticklabels=yticklabels,
                ax=ax)
    ax.set_title('data availability (black cells) for {}'.format(tableORpid))
    return fig, ax

def store_plot(result_file, figure):
    figure.tight_layout()
    figure.savefig(result_file)
    plt.close(figure)

In [ ]:
for table in tables:
    data_file = '{0}/180630/{1}.txt'.format(data_path, table)
    data1 = load_data(data_file)
    data_file = '{0}/180228/{1}.txt'.format(data_path, table)
    data2 = load_data(data_file)
    data = data1.add(data2, fill_value = 0)
    
    fig, ax = plot_data(data, table, 'auto', 'auto', None, None)
    result_file = '{0}/highlevel/{1}_highlevel.png'.format(result_path, table)
    store_plot(result_file, fig)
    
    fig, ax = plot_data(data, table, list(data.index.values), list(data.columns.values), 40, 30)
    result_file = '{0}/detailed/{1}_detailed.png'.format(result_path, table)
    store_plot(result_file, fig)

In [ ]:
for table in tables:
    data_file = '{0}/{1}.txt'.format(data_path, table)
    data = load_data(data_file)
    
    fig, ax = plot_data(data, table, 'auto', 'auto', None, None)
    result_file = '{0}/highlevel/{1}_highlevel.png'.format(result_path, table)
    store_plot(result_file, fig)
    
    fig, ax = plot_data(data, table, list(data.index.values), list(data.columns.values), 40, 30)
    result_file = '{0}/detailed/{1}_detailed.png'.format(result_path, table)
    store_plot(result_file, fig)

In [5]:
for pid in participants:
    
    pid_data = []
    for table in tables:
        data_file = '{0}/{1}.txt'.format(data_path, table)
        data = load_data(data_file)
        pid_data.append(data[pid])
    pid_data = pd.concat(pid_data, axis=1)
    pid_data.columns = tables
    
    fig, ax = plot_data(pid_data, pid, list(pid_data.index.values), list(pid_data.columns.values), 40, 10)
    result_file = '{0}/participants/pid{1}.png'.format(result_path, pid)
    store_plot(result_file, fig)